<a href="https://colab.research.google.com/github/desmond-rn/ComputationalLogic/blob/prolexa-plus/Assignment%20-%20Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prolexa Assignment Report 
This notebook serves are a short report for this 👉 [coursework assignment](https://github.com/desmond-rn/ComputationalLogic/blob/prolexa-plus/assignment.md). It goes thought our implementation of **Negation**, **Deault rule**, and **Existential quantification** by means of selected examples. In our implementations, we mostly added (instead of editing) to the preeixiting grammar and engine. For this demo, we will be using Prloxa Plus for its ease of use in a Notebook environment. For a more in-depth introduction to Prolexa and Prolexa Plus, see the [Prolexa Plus Demo Notebook](https://github.com/desmond-rn/ComputationalLogic/blob/prolexa-plus/Prolexa_Plus_Demo_Notebook.ipynb).


## Setup

Install SWI-Prolog

In [1]:
!apt-get install swi-prolog -qqq > /dev/null

Extracting templates from packages: 100%


Install Prolexa-Plus

In [2]:
%%capture
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/@neg -qqq > /dev/null

Instantiate Prolexa 


In [3]:
%%capture
!python -m nltk.downloader all      ## Download NLTK data because of changes to `meta_grammar`.

In [4]:
## For not displaying the output
%%capture   

from pyswip import Prolog
import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

The `respond()` utility function .

In [5]:
## @title Prolexa Interact

input = 'tell me all you know'  #@param {type:"string"}

def respond(input):
  first_answer = meta.standardised_query(pl, input)[0]['Output']
  print(first_answer)

respond(input)

b'I know nothing'


## Negation

This section covers our implementaiton of Negation in Prolexa. Let's solve the following example:
> Every teacher is happy. Donald is not happy. Therefore, Donald is not a teacher.

First, let's add adequate rules to Prolexa. 

In [6]:
# respond('forget all you know')

In [7]:
input = 'every teacher is happy'  #@param {type:"string"}
respond(input)

I will remember that every teacher is happy


In [8]:
input = 'Donald is not happy'  #@param {type:"string"}
respond(input)

I will remember that Donald is not happy


In [9]:
respond('tell me everything you know')

every teacher is happy. donald is not happy


Now let's test our implementation; and the way Prolexa get's to the result.

In [10]:
input = 'who is not a teacher'  #@param {type:"string"}
respond(input)

b'donald is not a teacher'


In [11]:
input = 'explain why donald is not a teacher'  #@param {type:"string"}
respond(input)

donald is not happy; every teacher is happy; therefore donald is not a teacher


Below, we can see that the same result is not obtained for other proper nouns initially in our grammar. This shows that default rule is not (yet) implemented in Prolexa's reasoning. 

In [12]:
input = 'explain why peter is not a teacher'  #@param {type:"string"}
respond(input)

Sorry, I don't think this is the case


### Changes to the grammar
In order for Prolexa to understang the sentences above, we needed to make a few changes to her grammar (see `prolexa_grammar.pl`). Speciffically, we added:
- the predicate `not` as seen in the lectures:
```
:- op(900, fy, not).
```

- the terms `donald`, `happy`, and `teacher` as proper noun, adjective, and noun respectively.

- additional clauses for `sentence1` and `verb_phrase` to parse negative sentences like 'Donald is not happy' in singular and plural form. Notice the Prolog goals incorporeated into DCG: 
```
sentence1(C) --> proper_noun(N,X),verb_phrase(N,T), {T=[(not X=>L)], C=[(not L:-true )]}.
verb_phrase(s,N) --> [is,not],property(s,M), {N=[(not M)]}.
verb_phrase(p,N) --> [are,not],property(p,M), {N=[(not M)]}.
```
- additional clause for negative questions:
```
question1(not P) --> [who],verb_phrase(s,Q),{Q=[(not _X=>P)]}.
```

### Changes to the engine

Next, we made changes to `prolexa_engine.pl`. Essentially, we reasonned by contraposition. Let P and Q be two propositions: proving that `P:-Q` is equivalent to proving that `not(Q):-not(P)`. This was acheived in Prolexa by adding the following clause to the existing `prove_rb` meta-interpreter:
```prolog
prove_rb(not B,Rulebase,P0,P):-
    find_clause((A:-B),Rule,Rulebase),
	prove_rb(not A,Rulebase,[p(not B,Rule)|P0],P).
```

## Defalut Rule

The goal here is to solve problems in the form:
> Most birds fly except penguins. Tweety is a bird. Therefore, assuming Tweety is not a penguin, Tweety flies.

Let's install and launch the prolexa branch that contains our default-rule implementation:

In [13]:
%%capture   

!yes | pip uninstall prolexa -qqq > /dev/null
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/@default-rule -qqq > /dev/null

# from pyswip import Prolog
# import prolexa.meta_grammar as meta

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

In [14]:
respond('forget all you know')

b'I am a blank slate'


In [15]:
input = 'all birds fly except penguins'  #@param {type:"string"}
respond(input)

I will remember that all birds fly except penguins


In [16]:
input = 'Tweety is a bird'  #@param {type:"string"}
respond(input)

I will remember that Tweety is a bird


In [17]:
respond('tell me everything you know')

all birds fly except penguins. tweety is a bird


Now let's test our implementation.

In [18]:
input = 'who flies'  #@param {type:"string"}
respond(input)

b'tweety flies'


In [19]:
# meta.reset_grammar()
input = 'explain why tweety flies'  #@param {type:"string"}
respond(input)

tweety is a bird; all birds fly except penguins; therefore tweety flies


When testing our implementation further, we see that when Tweety becomes part of the exeption (i.e. penguins), the default rule doesnt apply anymore.

In [20]:
input = 'Tweety is a penguin'  #@param {type:"string"}
respond(input)

I will remember that Tweety is a penguin


In [21]:
respond('tell me everything you know')

all birds fly except penguins. tweety is a bird. tweety is a penguin


In [22]:
input = 'explain why tweety flies'  #@param {type:"string"}
respond(input)

Sorry, I don't think this is the case


### Changes to the grammar

Our approach to implementing default rule was based on our solution for negation. All the changed made to the grammar and the engine for negation remain. The new additions to the grammar that are specific to default rule are:

- A new clause for `sentence1` to parse sentences containing the word `except`:
```
sentence1(C) --> determiner(N,M1,M2,_D),
                    noun(N,M1),
                    verb_phrase(N,M2), 
                    [except], 
                    noun(N,M3), 
                    {C=[(C1:-C2,not C3)], 
                          M1=(_X2=>C2), 
                          M2=(_X1=>C1), 
                          M3=(_X3=>C3)}.
```
- A few clause for `verb_phrase`:
```
verb_phrase(s,N) --> [does, not],iverb(p,M), {N=[(not M)]}.
verb_phrase(p,N) --> [do, not],iverb(p,M), {N=[(not M)]}.
```

### Changes to the engine

As for the engine, we added a default rules to the `prove-rb` meta-interpreter. These two rules are largelly base on the definition of `not` as implemented in 'negation as failure' (Simply-Logical, ch3).
```
prove_rb(not A,Rulebase,P0,P):-
    prove_rb(A,Rulebase,P0,P), !, fail.
prove_rb(not _A,_Rulebase,P,P):-!.
``` 

In conlcusion for this section, we can see that default-rule is a strong addition to Prolexa, but with consequences. The example below shows that Prolexa can now reach conclusions that she is unable to prove.

In [23]:
input = 'explain why peter does not fly'  #@param {type:"string"}
respond(input)

therefore peter does not fly


## Existential quantification

The goal in this final section is to solve problems in the form:
> Some humans are geniuses. Geniuses win prizes. Therefore, some humans win prizes.

The reasoning implemented in the previous sections remains avaialble here. Let's reinstall and launch the appropriate (default) branch:

In [24]:
%%capture   

!yes | pip uninstall prolexa -qqq > /dev/null
!yes | pip install git+https://github.com/desmond-rn/ComputationalLogic/ -qqq > /dev/null

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

In [25]:
respond('forget all you know')

b'I am a blank slate'


In [26]:
input = 'Some humans are geniuses'  #@param {type:"string"}
respond(input)

I will remember that Some humans are geniuses


In [27]:
input = 'Geniuses win prizes'  #@param {type:"string"}
respond(input)

I will remember that Geniuses win prizes


In [28]:
respond('tell me everything you know')

some humans are geniuses. some geniuses are human. every genius wins prizes


Notice that Prolexa Plus adds 'some geniuses are humans' in our rulebase. This was not the case for Prolexa. Fixing this issue could be part of some future work since it doesn't change our demonstration. Now let's test our implementation (the repetition in the explanation can be explored in a future work).

In [29]:
input = 'who wins prizes'  #@param {type:"string"}
respond(input)

b'some humans win prizes'


In [30]:
# meta.reset_grammar()
input = 'explain why some humans win prizes'  #@param {type:"string"}
respond(input)

some humans are geniuses; every genius wins prizes; some humans are geniuses; therefore some humans win prizes


### Changes to the grammar

The new additions to the grammar rules are:

- New predicates for 'genius' and 'win' declaring them as noun and (intrasitive) verb respectively:
```
pred(genius,   1,[n/genius]).
pred(win,     1,[v/win]).
```

- A new clause for `verb_phrase` in order to treat `win` as a transitive verb. This cheat declares `win` as an intransitive verb followed necesarily by the word 'prizes'. This was an adhoc solution in run to perform this demo as espected:
```
verb_phrase(N,M) --> iverb(N,M),[prizes].
```

- New clauses for `determiner` giving 'some' a "meaning" that can be written as a list of two clauses:
```
determiner(p, sk=>H1, sk=>H2, [(H1:-true),(H2:-true)]) -->[some].
determiner(p, sk=>H1, sk=>H2, [(H2:-true),(H1:-true)]) -->[some].
```

### Changes to the engine

Since we decided to treat the "meaning" of the determiner 'some' as a **list of two clauses**, this required extensive additions to the Prolexa engine:

- A new `prove_question` clause:
```
prove_question(Query,SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( prove_rb([Query,SecondQuery],Rulebase) ->
		transform([Query,SecondQuery],Clauses),
		phrase(sentence(Clauses),AnswerAtomList),
		atomics_to_string(AnswerAtomList," ",Answer)
	; Answer = 'Sorry, I don\'t think this is the case'
	).	
```

- Similarly, a new clause for `explain_question` needed for proofs:
```
explain_question([Query,SecondQuery],SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( prove_rb([Query,SecondQuery],Rulebase,[],Proof) ->
		maplist(pstep2message,Proof,Msg),
		phrase(sentence1([(Query:-true),(SecondQuery:-true)]),L),
		atomic_list_concat([therefore|L]," ",Last),
		append(Msg,[Last],Messages),
		atomic_list_concat(Messages,"; ",Answer)
	; Answer = 'Sorry, I don\'t think this is the case'
	).
```

- New clauses for `prove_rb`:
```
prove_rb([true,true],_Rulebase,P,P) :- !.
% prove_rb([A,true],Rulebase,P0,P) :- prove_rb(A,Rulebase,P0,P), !.
% prove_rb([true,A],Rulebase,P0,P) :- prove_rb(A,Rulebase,P0,P), !.
prove_rb([A,C],Rulebase,P0,P):-
	find_clause([(A:-B),D],Rule,Rulebase),
	(
		var(D) ->  		%%% D is uninstantiated
		prove_rb([B,C],Rulebase,[p(A,Rule)|P0],P) 
		; 
		D = (C:-E),
		prove_rb([B,E],Rulebase,[p(A,Rule),p(C,Rule)|P0],P)
	).
```

- Another set of clauses needed for proofs:
```
prove_rb([A,C],Rulebase,P0,P):-
	prove_rb((A,C),Rulebase,P0,P).
```

- At the heart of our approach, we devised a new strategy for finding body of clauses whose heads are known and unifiable; this with lists:
```
find_clause(Clause,Rule,[Rule|_Rules]):-
	Rule = [Rule1|Rule2],
	(
		Clause = [Clause1,Clause2] ->
		(
			copy_term([Rule1],[Clause1]) ->
			( Rule2 = [El|_] -> Clause2 = El ; true )
			; copy_term(Rule2,[Clause1]),
			Clause2 = Rule1
		)
		;
		(
			copy_term([Rule1],[Clause]) ->
			true
		; 	copy_term(Rule2,[Clause])
		)
	).
find_clause(Clause,Rule,[_Rule|Rules]):-
	find_clause(Clause,Rule,Rules).
```

- Finaly, A new clause to turn literrals in lists into clauses by adding `true` to their body:
```
transform([A,B],[(A:-true),(B:-true)]).
```

## Future work

Negation, default rule and existential quantification are all interresting aditions to Prolexa's reasoning capabilities. At the moment, most of these reaning techniques do not function together. For example, Prolexa is unable to parse the following sentence:
> Some humans are not happy

Combiniing these techniques will be part of our future work on this problem, along with implementing disjuction and abduction. 